In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ct.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1"], axis =1) 
fips_check.head() #10912

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,185624,3903327306,4/29/20,156,PPP,PROPARK INC,1 Union Place,Hartford,CT,6103.0,...,Parking Lots and Garages,999 Other,8129 Other Personal Services,8129.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"1 Union Place, Hartford, CT",41.768886,-72.681543
1,185625,4505207402,5/9/20,156,PPP,MONROE STAFFING SERVICES LLC,6 RESEARCH DR,SHELTON,CT,6484.0,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472
2,185626,5539227705,5/1/20,156,PPP,CENVEO WORLDWIDE LIMITED,200 First Stamford Place,Stamford,CT,6902.0,...,Stationery Product Manufacturing,322 Paper,3222 Converted Paper Product Manufacturing,3222.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385
3,185627,5630057006,4/6/20,156,PPP,SERVICE MANAGEMENT GROUP,25 CONTROLS DR,SHELTON,CT,6484.0,...,Janitorial Services,999 Other,5617 Services to Buildings and Dwellings,5617.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990
4,185628,8388727000,4/8/20,156,PPP,DAY PITNEY LLP,242 TRUMBULL ST,HARTFORD,CT,6103.0,...,Offices of Lawyers,999 Other,5411 Legal Services,5411.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,250 to 499,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan 

,LoanNumber,full_add,Lat,Long
5247,3539977109,"1486 highland ave building 2 unit 1, CHESHIRE, CT",NaN,NaN


In [7]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna #12892

,LoanNumber,full_add,Lat,Long
0,3903327306,"1 Union Place, Hartford, CT",41.768886,-72.681543
1,4505207402,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472
2,5539227705,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385
3,5630057006,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990
4,8388727000,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077
...,...,...,...,...
12888,9618168507,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780
12889,9648837000,"73 East St, PLAINVILLE, CT",41.669405,-72.861979
12890,9763078402,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417
12891,9854708505,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/CT_fips_scraped.csv") 
fips_check_scraped.head()

In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #40813

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #1239

<h4>FIPS Script

In [8]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('09')] #CT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [11]:
print(len(geo_id_list))

12892


Matching FIPS to DataFrame

In [12]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1239]

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-13-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,3903327306,"1 Union Place, Hartford, CT",41.768886,-72.681543,[090035021002009]
1,4505207402,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,[090011103022004]
2,5539227705,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,[090010223003001]
3,5630057006,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,[090011102023008]
4,8388727000,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,[090035021001038]
...,...,...,...,...,...
12888,9618168507,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,[090117053001004]
12889,9648837000,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,[090034205004006]
12890,9763078402,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,[090010739003009]
12891,9854708505,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,[090010106002002]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-14-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,3903327306,"1 Union Place, Hartford, CT",41.768886,-72.681543,090035021002009
1,4505207402,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,090011103022004
2,5539227705,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,090010223003001
3,5630057006,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,090011102023008
4,8388727000,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,090035021001038
...,...,...,...,...,...
12888,9618168507,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,090117053001004
12889,9648837000,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,090034205004006
12890,9763078402,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,090010739003009
12891,9854708505,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,090010106002002


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check

,LoanNumber,full_add,Lat,Long,FIPS_z
1457,1004577101,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3075,7637387710,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
10902,4931747305,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,3903327306,090035021002009
1,4505207402,090011103022004
2,5539227705,090010223003001
3,5630057006,090011102023008
4,8388727000,090035021001038
...,...,...
12888,9618168507,090117053001004
12889,9648837000,090034205004006
12890,9763078402,090010739003009
12891,9854708505,090010106002002


Read in base data

In [18]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ct.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/CT_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #15796


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,185624,185624,3903327306,4/29/20,156,PPP,PROPARK INC,1 Union Place,Hartford,CT,...,Parking Lots and Garages,999 Other,8129 Other Personal Services,8129.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"1 Union Place, Hartford, CT",41.768886,-72.681543
1,185625,185625,4505207402,5/9/20,156,PPP,MONROE STAFFING SERVICES LLC,6 RESEARCH DR,SHELTON,CT,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472
2,185626,185626,5539227705,5/1/20,156,PPP,CENVEO WORLDWIDE LIMITED,200 First Stamford Place,Stamford,CT,...,Stationery Product Manufacturing,322 Paper,3222 Converted Paper Product Manufacturing,3222.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385
3,185627,185627,5630057006,4/6/20,156,PPP,SERVICE MANAGEMENT GROUP,25 CONTROLS DR,SHELTON,CT,...,Janitorial Services,999 Other,5617 Services to Buildings and Dwellings,5617.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990
4,185628,185628,8388727000,4/8/20,156,PPP,DAY PITNEY LLP,242 TRUMBULL ST,HARTFORD,CT,...,Offices of Lawyers,999 Other,5411 Legal Services,5411.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,250 to 499,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12888,198512,198512,9618168507,3/12/21,156,PPS,DENISON PEQUOTSEPOS NATURE CENTER INC.,109 Pequotsepos Rd,Mystic,CT,...,All Other Miscellaneous Schools and Instruction,999 Other,6116 Other Schools and Instruction,6116.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780
12889,198513,198513,9648837000,4/9/20,156,PPP,GNAZZO FOOD CENTER INC,73 East St,PLAINVILLE,CT,...,Supermarkets and Other Grocery (except Conveni...,999 Other,4451 Grocery Stores,4451.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"73 East St, PLAINVILLE, CT",41.669405,-72.861979
12890,198514,198514,9763078402,2/17/21,156,PPS,ALL AROUND CONTRACTING CORP,76 Knowlton St,Bridgeport,CT,...,Other Heavy and Civil Engineering Construction,999 Other,2379 Other Heavy and Civil Engineering Constru...,2379.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417
12891,198515,198515,9854708505,3/12/21,156,PPS,WALKING WITH PURPOSE,15 E Putnam Ave,Greenwich,CT,...,All Other Publishers,511 512 519 Publishing,"5111 Newspaper, Periodical, Book, and Director...",5111.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677


Merge with MainFrame

In [19]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,185624,185624,3903327306,4/29/20,156,PPP,PROPARK INC,1 Union Place,Hartford,CT,...,999 Other,8129 Other Personal Services,8129.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"1 Union Place, Hartford, CT",41.768886,-72.681543,090035021002009
1,185625,185625,4505207402,5/9/20,156,PPP,MONROE STAFFING SERVICES LLC,6 RESEARCH DR,SHELTON,CT,...,999 Other,5613 Employment Services,5613.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,090011103022004
2,185626,185626,5539227705,5/1/20,156,PPP,CENVEO WORLDWIDE LIMITED,200 First Stamford Place,Stamford,CT,...,322 Paper,3222 Converted Paper Product Manufacturing,3222.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,090010223003001
3,185627,185627,5630057006,4/6/20,156,PPP,SERVICE MANAGEMENT GROUP,25 CONTROLS DR,SHELTON,CT,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,090011102023008
4,185628,185628,8388727000,4/8/20,156,PPP,DAY PITNEY LLP,242 TRUMBULL ST,HARTFORD,CT,...,999 Other,5411 Legal Services,5411.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,250 to 499,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,090035021001038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12888,198512,198512,9618168507,3/12/21,156,PPS,DENISON PEQUOTSEPOS NATURE CENTER INC.,109 Pequotsepos Rd,Mystic,CT,...,999 Other,6116 Other Schools and Instruction,6116.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,090117053001004
12889,198513,198513,9648837000,4/9/20,156,PPP,GNAZZO FOOD CENTER INC,73 East St,PLAINVILLE,CT,...,999 Other,4451 Grocery Stores,4451.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,090034205004006
12890,198514,198514,9763078402,2/17/21,156,PPS,ALL AROUND CONTRACTING CORP,76 Knowlton St,Bridgeport,CT,...,999 Other,2379 Other Heavy and Civil Engineering Constru...,2379.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,090010739003009
12891,198515,198515,9854708505,3/12/21,156,PPS,WALKING WITH PURPOSE,15 E Putnam Ave,Greenwich,CT,...,511 512 519 Publishing,"5111 Newspaper, Periodical, Book, and Director...",5111.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,090010106002002


If FIPS Encoded

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

In [20]:
fips_merge.loc[fips_merge["FIPS_z"].isnull()] #31

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
1457,187081,187081,1004577101,4/9/20,156,PPP,VISION FINANCIAL MARKETS LLC,120 Long Ridge Road 3 North,STAMFORD,CT,...,999 Other,5231 Securities and Commodity Contracts Interm...,5231.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3075,188699,188699,7637387710,5/1/20,156,PPP,PIONEER EDUCATION LLC,5200 Rte 42,Turnersville,CT,...,999 Other,6116 Other Schools and Instruction,6116.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
5247,190871,190871,3539977109,4/11/20,156,PPP,EM BENSON LLC,1486 highland ave building 2 unit 1,CHESHIRE,CT,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1486 highland ave building 2 unit 1, CHESHIRE, CT",NaN,NaN,NaN
10902,196526,196526,4931747305,4/30/20,156,PPP,ELANUS CAPITAL MANAGEMENT LLC,180 Varick Street,SALISBURY,CT,...,999 Other,5239 Other Financial Investment Activities,5239.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN


In [21]:
geo_CT_fips = fips_merge

Convert to File

In [22]:
geo_CT_fips.to_csv("data/state_data/geo/geo_fips/CT_fips_scraped.csv") 

Review & Compare

In [23]:
geo_CT_fips = pd.read_csv("data/state_data/geo/geo_fips/CT_fips_scraped.csv") 
geo_CT_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,185624,185624,3903327306,4/29/20,156,PPP,PROPARK INC,1 Union Place,Hartford,...,999 Other,8129 Other Personal Services,8129.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13
1,1,185625,185625,4505207402,5/9/20,156,PPP,MONROE STAFFING SERVICES LLC,6 RESEARCH DR,SHELTON,...,999 Other,5613 Employment Services,5613.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13
2,2,185626,185626,5539227705,5/1/20,156,PPP,CENVEO WORLDWIDE LIMITED,200 First Stamford Place,Stamford,...,322 Paper,3222 Converted Paper Product Manufacturing,3222.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13
3,3,185627,185627,5630057006,4/6/20,156,PPP,SERVICE MANAGEMENT GROUP,25 CONTROLS DR,SHELTON,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,500 or more,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13
4,4,185628,185628,8388727000,4/8/20,156,PPP,DAY PITNEY LLP,242 TRUMBULL ST,HARTFORD,...,999 Other,5411 Legal Services,5411.0,CONNECTICUT DISTRICT OFFICE,e $5-10 million,250 to 499,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12888,12888,198512,198512,9618168507,3/12/21,156,PPS,DENISON PEQUOTSEPOS NATURE CENTER INC.,109 Pequotsepos Rd,Mystic,...,999 Other,6116 Other Schools and Instruction,6116.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,9.011705e+13
12889,12889,198513,198513,9648837000,4/9/20,156,PPP,GNAZZO FOOD CENTER INC,73 East St,PLAINVILLE,...,999 Other,4451 Grocery Stores,4451.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,9.003421e+13
12890,12890,198514,198514,9763078402,2/17/21,156,PPS,ALL AROUND CONTRACTING CORP,76 Knowlton St,Bridgeport,...,999 Other,2379 Other Heavy and Civil Engineering Constru...,2379.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,9.001074e+13
12891,12891,198515,198515,9854708505,3/12/21,156,PPS,WALKING WITH PURPOSE,15 E Putnam Ave,Greenwich,...,511 512 519 Publishing,"5111 Newspaper, Periodical, Book, and Director...",5111.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,9.001011e+13


In [24]:
#fill in missing Lat, Long, FIPS
geo_CT_fips.loc[geo_CT_fips["FIPS_z"].isnull()] #1241

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
1457,1457,187081,187081,1004577101,4/9/20,156,PPP,VISION FINANCIAL MARKETS LLC,120 Long Ridge Road 3 North,STAMFORD,...,999 Other,5231 Securities and Commodity Contracts Interm...,5231.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3075,3075,188699,188699,7637387710,5/1/20,156,PPP,PIONEER EDUCATION LLC,5200 Rte 42,Turnersville,...,999 Other,6116 Other Schools and Instruction,6116.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
5247,5247,190871,190871,3539977109,4/11/20,156,PPP,EM BENSON LLC,1486 highland ave building 2 unit 1,CHESHIRE,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,CONNECTICUT DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1486 highland ave building 2 unit 1, CHESHIRE, CT",NaN,NaN,NaN
10902,10902,196526,196526,4931747305,4/30/20,156,PPP,ELANUS CAPITAL MANAGEMENT LLC,180 Varick Street,SALISBURY,...,999 Other,5239 Other Financial Investment Activities,5239.0,CONNECTICUT DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN
